# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [ ]:
#!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip

#download from my own google drive
!gdown --id 1ukfjOWyq_ZRhwMj01LJys6GhSvLEZnaF --output timit_11.zip
!unzip timit_11.zip
!ls 

Downloading...
From: https://drive.google.com/uc?id=1ukfjOWyq_ZRhwMj01LJys6GhSvLEZnaF
To: /content/timit_11.zip
372MB [00:05, 63.3MB/s]
Archive:  timit_11.zip
   creating: timit_11/
  inflating: timit_11/train_11.npy   
  inflating: timit_11/test_11.npy    
  inflating: timit_11/train_label_11.npy  
sample_data  timit_11  timit_11.zip


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [ ]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

Loading data ...


In [ ]:
#reshape for rnn:
# train = train.reshape(len(train),1,429)
# test = test.reshape(len(test),1,429)

print('Size of training data: {}'.format(train.shape))
print('Size of training label: {}'.format(train_label.shape))
print('Size of testing data: {}'.format(test.shape))

Size of training data: (1229932, 429)
Size of training label: (1229932,)
Size of testing data: (451552, 429)


## Create Dataset

In [ ]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [ ]:
VAL_RATIO = 0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 429)
Size of validation set: (245987, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [ ]:
BATCH_SIZE = 270  #enlarge batch size from 64 to 270 

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [ ]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

152

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [ ]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        # DNN
        self.layer1 = nn.Linear(429, 1024)
        self.layer2 = nn.Linear(1024, 2048)
        self.layer3 = nn.Linear(2048, 1024)
        self.layer4 = nn.Linear(1024, 512)
        self.layer5 = nn.Linear(512, 256)
        self.layer6 = nn.Linear(256, 128)
        self.layer7 = nn.Linear(128, 64)
        self.layer8 = nn.Linear(64, 32)
        self.out = nn.Linear(32, 39) 

        self.norm1 = nn.BatchNorm1d(1024)
        self.norm2 = nn.BatchNorm1d(2048)
        self.norm3 = nn.BatchNorm1d(1024)
        self.norm4 = nn.BatchNorm1d(512)
        self.norm5 = nn.BatchNorm1d(256)
        self.norm6 = nn.BatchNorm1d(128)
        self.norm7 = nn.BatchNorm1d(64)
        self.norm8 = nn.BatchNorm1d(32)
        
        self.drop = nn.Dropout(p=0.1)
        self.act_fn = nn.Sigmoid()


        # LSTM
        # input_size = 429
        # hidden_size = 128
        # self.lstm = nn.LSTM(input_size=input_size,hidden_size=hidden_size)
        # self.out = nn.Linear(128, 39)

    def forward(self, x):
        x = self.layer1(x)
        x = self.norm1(x)
        x = self.act_fn(x)
        x = self.drop(x)
      
        x = self.layer2(x)
        x = self.norm2(x)
        x = self.act_fn(x)
        x = self.drop(x)
        
        x = self.layer3(x)
        x = self.norm3(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.layer4(x)
        x = self.norm4(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.layer5(x)
        x = self.norm5(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.layer6(x)
        x = self.norm6(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.layer7(x)
        x = self.norm7(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.layer8(x)
        x = self.norm8(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.out(x)

        # outs,(h,c) = self.lstm(x)
        # x = self.out(outs)
        
        return x

## Training

In [ ]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [ ]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [ ]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 100               # number of training epoch
learning_rate = 0.0001           # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0)

DEVICE: cuda


In [ ]:
# start training

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        #inputs = inputs.unsqueeze(dim=0)
        #print(inputs.shape)
        outputs = model(inputs)
        #outputs = outputs.squeeze(dim=0)
        #print(outputs.shape,labels.shape)
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward()
        optimizer.step()

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                #inputs = inputs.unsqueeze(dim=0)
                #print(inputs.shape)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')

[001/100] Train Acc: 0.302612 Loss: 2.784786 | Val Acc: 0.406643 loss: 2.252640
saving model with acc 0.407
[002/100] Train Acc: 0.413709 Loss: 2.137289 | Val Acc: 0.470143 loss: 1.829015
saving model with acc 0.470
[003/100] Train Acc: 0.456493 Loss: 1.882468 | Val Acc: 0.505262 loss: 1.637715
saving model with acc 0.505
[004/100] Train Acc: 0.486010 Loss: 1.756126 | Val Acc: 0.549395 loss: 1.523748
saving model with acc 0.549
[005/100] Train Acc: 0.511801 Loss: 1.673272 | Val Acc: 0.574660 loss: 1.443622
saving model with acc 0.575
[006/100] Train Acc: 0.530840 Loss: 1.609792 | Val Acc: 0.595267 loss: 1.378122
saving model with acc 0.595
[007/100] Train Acc: 0.546011 Loss: 1.555776 | Val Acc: 0.610573 loss: 1.319429
saving model with acc 0.611
[008/100] Train Acc: 0.559069 Loss: 1.508752 | Val Acc: 0.623907 loss: 1.271090
saving model with acc 0.624
[009/100] Train Acc: 0.571239 Loss: 1.467799 | Val Acc: 0.632281 loss: 1.236102
saving model with acc 0.632
[010/100] Train Acc: 0.58205

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [ ]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        # post processing steps
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        
        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))

# Vote from numerous models' prediction:

In [ ]:
# Vote classifier:
# model1 (DNN 3 layers) test acc: 0.72860 --> df1
# model2 (DNN 5 layers) test acc: 0.73077 --> df2
# model3 (DNN 8 layers) test acc: 0.72898 --> df3
import io
from google.colab import files
import pandas as pd

uploaded = files.upload()
uploaded2 = files.upload()
uploaded3 = files.upload()

Saving pred.csv to pred.csv


Saving pred2.csv to pred2.csv


Saving pred3.csv to pred3.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['pred.csv']))
df2 = pd.read_csv(io.BytesIO(uploaded2['pred2.csv']))
df3 = pd.read_csv(io.BytesIO(uploaded3['pred3.csv']))

In [ ]:
from collections import Counter
tmp = []
for i in range(len(df)):
  cnt = Counter([df.iloc[i][1],df2.iloc[i][1],df3.iloc[i][1]])
  # 三個預測都不一樣以model2為準
  if len(cnt) == 3:     
    tmp.append(df2.iloc[i][1])
  # 取預測次數最多的
  elif len(cnt) == 2:
    #print(cnt)
    k = cnt.most_common(1)[0][0]
    #print(k)
    tmp.append(k)
  # 三個預測都一致
  else:
    tmp.append(df.iloc[i][1])

Streaming output truncated to the last 5000 lines.
<class 'collections.Counter'>
Counter({7: 2, 9: 1})
7
<class 'collections.Counter'>
Counter({7: 2, 9: 1})
7
<class 'collections.Counter'>
<class 'collections.Counter'>
<class 'collections.Counter'>
Counter({14: 2, 17: 1})
14
<class 'collections.Counter'>
Counter({17: 2, 14: 1})
17
<class 'collections.Counter'>
Counter({17: 2, 14: 1})
17
<class 'collections.Counter'>
Counter({17: 2, 14: 1})
17
<class 'collections.Counter'>
<class 'collections.Counter'>
<class 'collections.Counter'>
Counter({1: 2, 17: 1})
1
<class 'collections.Counter'>
<class 'collections.Counter'>
Counter({17: 2, 1: 1})
17
<class 'collections.Counter'>
<class 'collections.Counter'>
<class 'collections.Counter'>
<class 'collections.Counter'>
Counter({35: 2, 31: 1})
35
<class 'collections.Counter'>
Counter({35: 2, 31: 1})
35
<class 'collections.Counter'>
Counter({31: 2, 35: 1})
31
<class 'collections.Counter'>
Counter({31: 2, 35: 1})
31
<class 'collections.Counter'>
Coun

In [ ]:
with open('vote_prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(tmp):
        f.write('{},{}\n'.format(i, y))